# ME TIENE QUE SALIR

In [1]:
from kan import *
import torch
import numpy as np
import pandas as pd
torch.set_default_dtype(torch.float64)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [2]:
dataset = pd.read_csv('/home/jesus_gh/MetodosBiomAI/KAN-RXNFP/schneider50k.tsv',sep='\t')
dataset['superclass'] = dataset['rxn_class'].apply(lambda x: int(str(x).split('.')[0]) - 1)
#print(dataset.head())

In [3]:
# Lista de columnas a eliminar
columnas_a_eliminar = ['source', 'split',]  

# Eliminar las columnas del dataset original
dataset.drop(columns=columnas_a_eliminar, inplace=True)

# Mostrar las primeras filas del dataset modificado
dataset.head()

,Unnamed: 0,original_rxn,rxn_class,rxn,superclass
0,0,[CH3:17][S:14](=[O:15])(=[O:16])[N:11]1[CH2:10...,6.1.5,C1CCCCC1.CCO.CS(=O)(=O)N1CCN(Cc2ccccc2)CC1.[OH...,5
1,1,O.O.[Na+].[CH3:1][c:2]1[cH:7][c:6]([N+:8](=O)[...,7.1.1,CCOC(C)=O.Cc1cc([N+](=O)[O-])ccc1NC(=O)c1ccccc...,6
2,2,[CH3:1][O:2][c:3]1[cH:4][cH:5][c:6](-[c:9]2[cH...,1.8.5,COc1ccc(-c2coc3ccc(-c4nnc(S)o4)cc23)cc1.COc1cc...,0
3,3,Cl.[CH3:43][CH2:42][S:44](=[O:45])(=[O:46])Cl....,2.2.3,CCS(=O)(=O)Cl.CN(C(=O)N(C)[C@@H]1CN(C(=O)C2CCN...,1
4,4,[CH3:25][O:24][c:21]1[cH:22][cH:23][c:17]([O:1...,1.3.7,COc1ccc(OC)c(N)c1.Cc1cc(Cl)nc(-c2ccccn2)n1>>CO...,0


In [4]:
dataset = dataset.sample(150)  # Usar un subconjunto pequeño para probar

In [5]:
inputs = dataset['rxn'] #Indico cuales son las entradas
outputs  = dataset['superclass'] #Indico cuales son las salidas

In [6]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Crear una lista de caracteres únicos encontrados en SMILES
smiles_vocab = list(set("".join(dataset['rxn'])))
max_length = dataset['rxn'].str.len().max()  # Longitud máxima de SMILES

# Tokenización con padding
def one_hot_encode_smiles(smiles, vocab, max_len):
    vocab_size = len(vocab)
    char_to_idx = {char: idx for idx, char in enumerate(vocab)}
    encoding = np.zeros((max_len, vocab_size))  # Matriz con padding
    for i, char in enumerate(smiles[:max_len]):  # Truncar si es necesario
        encoding[i, char_to_idx[char]] = 1
    return encoding.flatten()

# Aplicar la codificación one-hot con padding a todas las entradas SMILES
inputs_tokenized = dataset['rxn'].apply(lambda x: one_hot_encode_smiles(x, smiles_vocab, max_length)).tolist()

# Convertir las listas tokenizadas en tensores
inputs_tensor = torch.tensor(inputs_tokenized, dtype=torch.float64, device=device)


/tmp/ipykernel_446637/3186771519.py:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  inputs_tensor = torch.tensor(inputs_tokenized, dtype=torch.float64, device=device)


In [7]:
from sklearn.model_selection import train_test_split
import torch

# Convertir etiquetas de superclases en tensores
outputs_tensor = torch.tensor(outputs.values, dtype=torch.float64, device=device)

# Dividir en conjuntos de entrenamiento y prueba
train_inputs, test_inputs, train_outputs, test_outputs = train_test_split(inputs_tensor, outputs_tensor, test_size=0.3, random_state=42)

# Imprimo las dimensiones de los conjuntos de entrenamiento y prueba
print(f'Train set: {train_inputs.shape}, {train_outputs.shape}') 
print(f'Test set: {test_inputs.shape}, {test_outputs.shape}')


Train set: torch.Size([105, 13120]), torch.Size([105])
Test set: torch.Size([45, 13120]), torch.Size([45])


In [8]:
# Imprimir los tipos de datos de los tensores
print(f"Train input dtype: {train_inputs.dtype}")
print(f"Test input dtype: {test_inputs.dtype}")

Train input dtype: torch.float64
Test input dtype: torch.float64


In [9]:
# Hago un diccionario con los conjuntos de datos
dataset_kan = {
    'train_input': train_inputs,
    'train_label': train_outputs,
    'test_input': test_inputs,
    'test_label': test_outputs,
}

In [10]:
# Inicializo el modelo KAN, Una entrada, 
# una capa oculta de 5 neuronas, dos neuranas multiples ocultas y una salida
model = KAN(width=[train_inputs.shape[1], [5,2], 1], grid=3, k=3)

checkpoint directory created: ./model
saving model version 0.0


# Definir una métrica de precisión para entrenamiento y prueba
def train_acc():
    return torch.mean((torch.round(model(train_inputs)) == train_outputs).float()) # Aproximación de la precisión
# round redondea a 0 o 1


def test_acc():
    return torch.mean((torch.round(model(test_inputs)) == test_outputs).float())

#Luego probar con argmax en vez de torch

In [11]:
# Definir las funciones de métrica personalizadas
# argmax nos da el indice del valor maximo de la fila, lo que nos da la clase
def train_acc():
    predictions = model(train_inputs)
    return torch.mean((torch.argmax(predictions, dim=1) == train_outputs).float()) 

def test_acc():
    predictions = model(test_inputs)
    return torch.mean((torch.argmax(predictions, dim=1) == test_outputs).float())

In [12]:
train_outputs = train_outputs.view(-1, 1) # Asegurarse de que las etiquetas tengan la forma correcta

In [13]:
results = model.fit(dataset_kan, opt="LBFGS",steps=50, metrics=(train_acc, test_acc)) # Entrenar el modelo
results['train_acc'][-1], results['test_acc'][-1] # Imprimir la precisión del modelo


description:   0%|                                                           | 0/50 [00:00<?, ?it/s]/home/jesus_gh/miniconda3/envs/KAN_rxnfp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
| train_loss: 3.27e+00 | test_loss: 3.30e+00 | reg: 1.78e+02 | : 100%|█| 50/50 [29:47<00:00, 35.75s/


saving model version 0.1


(0.2857142984867096, 0.3333333432674408)

In [14]:
print(f"Final train accuracy: {results['train_acc'][-1]}")
print(f"Final test accuracy: {results['test_acc'][-1]}")

Final train accuracy: 0.2857142984867096
Final test accuracy: 0.3333333432674408


In [ ]:
model.plot()


In [ ]:
lib = ['x','x^2']

model.auto_symbolic(lib=lib)

In [ ]:
formula1, formula2 = model.symbolic_formula()[0]
formula1

In [18]:
model = model.prune()


saving model version 0.2


In [19]:
model.plot()

: 

In [67]:
from kan.utils import ex_round

ex_round(model.symbolic_formula()[0][0],4)

0